# Hello-World DSPy Agent

> A minimal DSPy agent implementation to echo or compute simple inputs.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import dspy
import os

# Configure a default LLM - using gpt-4o-mini as specified in the plan
# Note: In real usage, you'd set OPENAI_API_KEY in your environment
default_lm = dspy.LM("openai/gpt-4o-mini")
dspy.configure(lm=default_lm)

class Echo(dspy.Module):
    """Echoes the input message back."""
    def forward(self, message: str) -> str:
        return message

In [ ]:
#| export
def make_hello_agent(llm=None):
    """Create a simple DSPy agent with the Echo tool and LLM capabilities.
    
    Args:
        llm: A DSPy language model. If None, uses the default configured LM.
    
    Returns:
        A HelloAgent instance that can process messages using the Echo tool and LLM.
    """
    # Use provided LLM or the default one
    lm = llm or dspy.settings.lm
    
    # Create a simple signature for LLM-based response generation
    signature = dspy.Signature(
        "message -> response",
        "Generate a friendly response to the input message."
    )
    
    # Create an Agent class that can use Echo tool and LLM
    class HelloAgent(dspy.Module):
        def __init__(self):
            super().__init__()
            self.tools = [Echo()]
            # Use ChainOfThought module to get LLM reasoning
            self.predictor = dspy.ChainOfThought(signature)
            
        def forward(self, message):
            # First use the Echo tool
            echo_result = self.tools[0].forward(message)
            
            # Then get an LLM response
            llm_result = self.predictor(message=message)
            
            # Return both results
            return {
                "echo_result": echo_result,
                "llm_response": llm_result.response,
                "reasoning": getattr(llm_result, "reasoning", None)
            }
            
    # Return a new instance of the HelloAgent
    return HelloAgent()

In [ ]:
# Demo the Hello-World agent with LLM capabilities
agent = make_hello_agent()
message = "Hello, DSPy! What can you do with Semantic Web data?"
result = agent(message)

print("Echo result:", result["echo_result"])
print("\nLLM response:", result["llm_response"])
print("\nLLM reasoning:", result["reasoning"])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()